In [ ]:
import os
import fnmatch
import matplotlib
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

%matplotlib inline
cmap = matplotlib.cm.get_cmap("tab20")

In [ ]:
# Setting filepaths
#
# Get Current Working Directory
CWD = os.getcwd()

# Set Relative and Absolute filepaths for clean data
CLEAN_DATA_RELPATH = os.path.join(os.pardir, os.pardir, "data", "clean")
CLEAN_DATA_ABSPATH = os.path.join(CWD, CLEAN_DATA_RELPATH)

In [ ]:
# Find data files
datafiles = fnmatch.filter(os.listdir(CLEAN_DATA_ABSPATH), "*.csv")

In [ ]:
# Read in the first datafile
filepath = os.path.join(CLEAN_DATA_ABSPATH, datafiles[0])
data = pd.read_csv(
    filepath,
    converters={
        "dt_start": pd.to_datetime,
        "dt_end": pd.to_datetime,
        "Duration": pd.to_timedelta,
    },
    na_values=0.0,
)

# Sort the data by start date
data.sort_values("dt_start", inplace=True, ignore_index=True)

# Convert `Duration` into hours
data["Duration"] = data["Duration"] / np.timedelta64(1, "h")

### Total time spent on tasks per week

In [ ]:
# Calculate total duration per task per week number
duration_per_task = pd.DataFrame({})
for task in np.sort(data["Task"].dropna().unique()):
    duration_per_task[task] = (
        data[data["Task"] == task].groupby("week_number")["Duration"].sum()
    )

duration_per_task["Total"] = data.groupby("week_number")["Duration"].sum()
duration_per_task = duration_per_task.T

In [ ]:
@interact
def make_pie_chart_week(column=duration_per_task.columns.tolist()):
    duration_per_task[column].drop("Total", axis=0).dropna().plot.pie(
        subplots=True,
        figsize=(12, 8),
        title=f"Total = {duration_per_task.loc['Total', column]:.1f} hrs",
        legend=False,
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    );

### Total time spent on projects per week

In [ ]:
duration_per_project = pd.DataFrame({})

for project in np.sort(data["Project"].dropna().unique()):
    duration_per_project[project] = (
        data[data["Project"] == project].groupby("week_number")["Duration"].sum()
    )

duration_per_project["Total"] = data.groupby("week_number")["Duration"].sum()
duration_per_project = duration_per_project.T.sort_index()

In [ ]:
@interact
def make_pie_chart_project(column=duration_per_project.columns.tolist()):
    duration_per_project[column].drop("Total", axis=0).dropna().plot.pie(
        subplots=True,
        figsize=(12, 8),
        title=f"Total = {duration_per_project.loc['Total', column]:.1f} hrs",
        legend=False,
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    );

### Total time spent on tasks per project (across all 3 weeks)

In [ ]:
columns = np.sort(data["Task"].dropna().unique())
project_df = pd.DataFrame(columns=columns)

for project in np.sort(data["Project"].dropna().unique()):
    project_df.loc[project] = (
        data[data["Project"] == project].groupby("Task")["Duration"].sum()
    )

project_df["Total"] = project_df.sum(axis=1)
project_df = project_df.T

In [ ]:
@interact
def make_pie_chart(column=project_df.columns.tolist()):
    project_df[column].drop("Total", axis=0).dropna().plot.pie(
        subplots=True,
        figsize=(12, 8),
        title=f"Total = {project_df.loc['Total', column]:.1f} hrs",
        legend=False,
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    );